In [1]:
from tqdm import tqdm 

from transformers import T5TokenizerFast

from tokenizers import ByteLevelBPETokenizer
from tokenizers import AddedToken, Regex, Tokenizer, decoders, normalizers, pre_tokenizers, trainers
from tokenizers.processors import TemplateProcessing
from mecab import MeCab

In [2]:
special_tokens = ["[PAD]", "[BOS]", "[EOS]", "[UNK]"]

special_token_dict = {}

for i in range(len(special_tokens)):
    special_token_dict[special_tokens[i][1:-1]] = {
        "id" : i,
        "token" : special_tokens[i]
    }

special_token_dict

{'PAD': {'id': 0, 'token': '[PAD]'},
 'BOS': {'id': 1, 'token': '[BOS]'},
 'EOS': {'id': 2, 'token': '[EOS]'},
 'UNK': {'id': 3, 'token': '[UNK]'}}

In [3]:
special_tokens = special_token_dict
special_tokens_list = [None] * len(special_tokens)
for token_dict in special_tokens.values():
    special_tokens_list[token_dict["id"]] = token_dict["token"]

In [4]:
TemplateProcessing_dict = {
    "single" : "$A {}".format(special_token_dict['EOS']['token']),
    "special_tokens" : [
        (special_token_dict['BOS']['token'], special_token_dict['BOS']['id']),
        (special_token_dict['EOS']['token'], special_token_dict['EOS']['id']),
    ]
}

In [5]:
norm_keys = ['Nmt', 'NFKC', 'Replace', 'Lowercase']

In [6]:
normalizers_dict = {
    'Nmt' : normalizers.Nmt(),
    'NFKC' : normalizers.NFKC(),
    'Replace' : normalizers.Replace(Regex(" {2,}"), " "),
    'Lowercase' : normalizers.Lowercase(),
}

In [7]:
tokenizer = ByteLevelBPETokenizer()

In [8]:
tokenizer.normalizer = normalizers.Sequence(
    [ normalizers_dict[key] for key in norm_keys]
)

In [9]:
tokenizer.post_processor = TemplateProcessing(**TemplateProcessing_dict)


In [10]:
tokenizer.train(files=["namuwiki_mecab.txt"], vocab_size=32_000, min_frequency=2, special_tokens=special_tokens_list)

In [11]:
t5_tokenizer = T5TokenizerFast(tokenizer_object=tokenizer)

In [12]:
t5_tokenizer.tokenize("あｆ")

['ã', 'ģ', 'Ĥ', 'f']

In [13]:
text = '''
그 이외에도 맵을 훨씬 화려하게 개편하거나, 주요 npc들의 배틀 장면에 애니메이션을 추가함으로써 훨씬 게임이 다채로워졌다. 또한 컨텐츠도 많이 늘어나서 즐길거리가 엄청나게 많아졌다.

특히 메인 스토리의 부실은 DP의 가장 큰 문제점으로 지적받았는데 최종보스 태홍의 비중을 늘리고, 그의 목적과 과거에 대해 훨씬 더 자세히 설명하며, 핸섬, 플루토 등의 주요 캐릭터를 더 추가하여 메인 스토리의 재미를 더 부가했다.
'''.strip()

In [14]:
tokenizer.decode(t5_tokenizer.encode(text))

'그 이외에도 맵을 훨씬 화려하게 개편하거나, 주요 npc들의 배틀 장면에 애니메이션을 추가함으로써 훨씬 게임이 다채로워졌다. 또한 컨텐츠도 많이 늘어나서 즐길거리가 엄청나게 많아졌다. 특히 메인 스토리의 부실은 dp의 가장 큰 문제점으로 지적받았는데 최종보스 태홍의 비중을 늘리고, 그의 목적과 과거에 대해 훨씬 더 자세히 설명하며, 핸섬, 플루토 등의 주요 캐릭터를 더 추가하여 메인 스토리의 재미를 더 부가했다.'

In [15]:
t5_tokenizer.encode("안녕하세요")

[30950, 489, 11520, 2]

In [16]:
t5_tokenizer.encode("おはようございます！")

[163,
 227,
 236,
 163,
 227,
 111,
 163,
 228,
 234,
 163,
 227,
 232,
 163,
 6034,
 163,
 227,
 248,
 163,
 227,
 230,
 163,
 227,
 126,
 163,
 227,
 251,
 4,
 2]

In [17]:
tokenizer = t5_tokenizer

In [18]:
num_sentinel_tokens = 100

sentinel_list = []
for i in range(num_sentinel_tokens):
    sentinel_list.append('[sentinel_{}]'.format(i))

In [19]:
sentinel_list.extend(['[R]','[X]','[S]','[SEP]'])

In [20]:
special_tokens_dict = {'additional_special_tokens': sentinel_list}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

In [21]:
tokenizer.pad_token_id = 0
tokenizer.bos_token_id = 1
tokenizer.eos_token_id = 2
tokenizer.unk_token_id = 3
tokenizer.sep_token_id = 32103

In [23]:
len(tokenizer)

32104

In [24]:
tokenizer_save_path = '../namuwiki_ul2_tokenizer'

In [25]:
tokenizer.save_pretrained(tokenizer_save_path)

('../namuwiki_ul2_tokenizer/tokenizer_config.json',
 '../namuwiki_ul2_tokenizer/special_tokens_map.json',
 '../namuwiki_ul2_tokenizer/tokenizer.json')

In [26]:
test_tokenizer =  tokenizer.from_pretrained(tokenizer_save_path)

In [27]:
len(test_tokenizer)

32104